# Singular Value Decomposition (SVD)

##### 2.c: evaluate the reconstruction performance on at least two IPSP datasets that were not part of the training

1. Load test data sets
2. Load the truncated left singular vectors $U_{reduced}$
3. Transform the test data into the reduced state with $$ X_{test,reduced} = U_{reduced}^T X_{test} $$
4. Reconstruct the reduced test data $ X_{test,reduced} $ with $$ X_{test,reconstr} = U_{reduced} X_{test,reduced}$$
5. Compare the reconstructed test set with the original test set using appropriate metrics, such as MSE $$ MSE = mean((X_{test} - X_{test,reconstr})^2) $$

In [5]:
import torch as pt
from torch import flatten
from os.path import join

output_path = "../output/SVD"
data_path = "../data"

test_keys = ['ma0.84_alpha5.00', 'ma0.84_alpha4.50']

#### 1. Load test datasets

In [12]:
cp_084_data = pt.load(join(data_path, "cp_084_500snaps.pt"))
X_test_1= cp_084_data[test_keys[0]].flatten(0, 1)
X_test_2 = cp_084_data[test_keys[1]].flatten(0, 1)

X_test_1.shape
X_test_2.shape

torch.Size([73935, 500])

#### 2. Load the truncated left singular vectors $U_{reduced}$

In [13]:
# load the left singular vectors
U_90 = pt.load(join(output_path, "U_90.pt"))
U_opt = pt.load(join(output_path, "U_opt.pt"))

U_90_T = pt.transpose(U_90, 0, 1)
print(U_90_T.shape)

torch.Size([2076, 73935])


#### 3. Transform the test data into the reduced state with $$ X_{test,reduced} = U_{reduced}^T X_{test} $$

In [24]:
X_test_1_reduced = pt.transpose(U_opt, 0, 1) @ X_test_1

X_test_2_reduced = pt.transpose(U_opt, 0, 1) @ X_test_2 

#### 4. Reconstruct the reduced test data $ X_{test,reduced} $ with $$ X_{test,reconstr} = U_{reduced} X_{test,reduced}$$

In [25]:
X_test_1_reconstr = U_opt @ X_test_1_reduced
X_test_1_reconstr.shape

X_test_2_reconstr = U_opt @ X_test_2_reduced
X_test_2_reconstr.shape

torch.Size([73935, 500])

#### 5. Compare the reconstructed test set with the original test set using appropriate metrics, such as MSE $$ MSE = mean((X_{test} - X_{test,reconstr})^2) $$

In [34]:
MSE_1 = pt.mean((X_test_1 - X_test_1_reconstr) ** 2)
print("The MSE between the original test data 1 and the reconstructed test data 1 is:   ", round(MSE_1.item(), 5))

MSE_2 = pt.mean((X_test_2 - X_test_2_reconstr) ** 2)
print("The MSE between the original test data 1 and the reconstructed test data 1 is:   ", round(MSE_2.item(), 5))

The MSE between the original test data 1 and the reconstructed test data 1 is:    0.01967
The MSE between the original test data 1 and the reconstructed test data 1 is:    0.019


#### or the Variance reconstruction in %

In [35]:
total_variance = pt.sum((X_test_1 - pt.mean(X_test_1)) ** 2)
residual_variance = pt.sum((X_test_1 - X_test_1_reconstr) ** 2)
explained_variance = total_variance - residual_variance

percentage_reconstruction = (explained_variance / total_variance) * 100
print("For test data 1, ", round(percentage_reconstruction.item(), 2), "% of variance could be reconstructed")

total_variance = pt.sum((X_test_2 - pt.mean(X_test_2)) ** 2)
residual_variance = pt.sum((X_test_2 - X_test_2_reconstr) ** 2)
explained_variance = total_variance - residual_variance

percentage_reconstruction = (explained_variance / total_variance) * 100
print("For test data 2, ", round(percentage_reconstruction.item(), 2), "% of variance could be reconstructed")

For test data 1,  89.86 % of variance could be reconstructed
For test data 2,  89.82 % of variance could be reconstructed
